# Algorex HCC Library
This notebook provides a demonstration of the Algorex HCC library. In developing this library we priortized three main features:

1. Return an accurate set of risk scores using the HCC Algorithim for all available models. 
2. Provide easy inputs so that the library can be integrated in a range of analytical or other applications. 
3. Give analyst/developer a rich interface into the underlying mechanics of the risk adjusment algorithim. Whether to the codes, their mappings, and their coefficients. 


In [1]:
from bokeh.plotting import figure, output_notebook, show, output_file
from bokeh.palettes import Blues, BuGn, viridis, Paired, plasma, PuBuGn
from bokeh.models import FixedTicker, FactorRange,Color,CustomJS, HoverTool,CategoricalAxis, LabelSet, Label, ColumnDataSource, widgets,CategoricalColorMapper,LinearInterpolator, LinearColorMapper, LogColorMapper
from bokeh.models import (GMapPlot, GMapOptions, Range1d, PanTool, WheelZoomTool, BoxSelectTool, HoverTool,  ResetTool, ZoomInTool, ZoomOutTool)
from bokeh.layouts import row, column, widgetbox
from bokeh.models.glyphs import Patches, Line, Circle
from bokeh.charts import Histogram, output_file, show, Bar, color,  Scatter
from bokeh.resources import CDN
import sklearn
from sklearn.utils.validation import check_array
import random
import squarify
import pandas as pd
import numpy as np


output_notebook()


%load_ext sql
%sql sqlite:///mit-poster.db
%sql ATTACH '../jupyterdemo/claims.db' as mem;


Loading BokehJS ...

Done.


[]

In [2]:
import AlgorexCore as rex
from hcc import *
cvars= community_regression()

bottom


## Loading Data
For the use of the demonstration, we are using a deidentifed claims data warehouse embedded in this notebook for the tutorial. This simulates exactly how you can integrate this library with other data warehouses or databases. 

In the cell below, we are selecting the diagnosis codes for one patient in one year and then we select the same patient in the next year. 



In [3]:
yr1_diags = %sql SELECT DISTINCT Diagnosis  From diagnoses  where PatientID = '0220F11E0B2EC004' and ClaimFromDate like '2008%';
yr2_diags = %sql SELECT DISTINCT Diagnosis  From diagnoses  where PatientID = '0220F11E0B2EC004' and ClaimFromDate like '2009%';

Done.
Done.


To load the data into the HCC library, you create beneficiary objects and add the diagnosis codes. Our example patient has **{{len(yr1_diags)}}** seperate diagnosis codes to be added in year 1 and **{{len(yr2_diags)}}** in year 2. 

The same use case can be used for whenever users have diagnosis codes from multiple sources such as from:
* at-home assesments from a third-party
* mined medical records
* other sources based on outreach efforts.

The library allows the developer/analyst to compare the output of the sources and compare the uplift(or downlift). 

Our example patient here is being named 'Jane'. 


In [4]:
jane = Beneficiary(hicno='0220F11E0B2EC004', sex='female', dob='19480601')
jane_alt = Beneficiary(hicno='0220F11E0B2EC004', sex='male', dob='19480601')

#antonio.add_diagnosis(Diagnosis(antonio,"49320",ICDType.NINE))


for icd9 in yr1_diags:
    code = icd9[0]
    #print(type(code), code)
    jane.add_diagnosis(Diagnosis(jane,code, int(ICDType.NINE)))

for icd9 in yr1_diags + yr2_diags:
    code = icd9[0]
    #print(type(code), code)
    jane.add_diagnosis(Diagnosis(jane_alt,code, int(ICDType.NINE)))

The library can output scores across all of the CMS-HCC models including instituional, new enrollee, and community rated groups. For 2017, there will be nine seperate models and will update to allow for that. 

In [5]:
beneficiary_has_hcc(jane, X)

print(score(jane,X,Score))
print(score(jane_alt,X,Score))

X             | Score             
--------------|-------------------
institutional | 12.458000000000002
community     | 17.212999999999994
X             | Score             
--------------|-------------------
community     | 21.09             
institutional | 14.851999999999997


## Understand the Mechanics

Getting the score is just one data point, what about how that score worked? the HCC models consists over close to 190 disease categories and complex disease interactions. With our library, the analyst can access the underlying coefficients that are used for this patient. 

In the example below, we are using our sample patient 'Jane' and asking the system to obtain all the indicator variables this patient is eligible. We can also see all the parts that lead up to this score. 

![diagram](./diagram.png)

The query below is based on this idea. Identify all indicators and underlying categories and codes that are true for this patient. 

In [6]:

results = indicator(jane,CC) & beneficiary_icd(jane,ICD,Type)  & CC.in_(cvars) & coefficient("CE_"+CC,Coef)   
print(results[0:30])

[('DIABETES_CHF', '9597', 9, 0.237), ('HCC170', '25003', 9, 0.363), ('HCC134', '496', 9, 0.617), ('CHF_COPD', '2512', 9, 0.255), ('COPD_CARD_RESP_FAIL', '2766', 9, 0.42), ('HCC186', 'V693', 9, 1.032), ('CHF_RENAL', '2761', 9, 0.201), ('HCC2', '40490', 9, 0.52), ('HCC21', '2767', 9, 0.653), ('HCC40', '8921', 9, 0.376), ('CHF_RENAL', '27652', 9, 0.201), ('HCC23', '27652', 9, 0.24), ('HCC29', '70715', 9, 0.313), ('HCC115', '7905', 9, 0.212), ('HCC111', '9058', 9, 0.388), ('CANCER_IMMUNE', '7891', 9, 1.101), ('HCC84', '2720', 9, 0.326), ('HCC2', '2639', 9, 0.52), ('HCC21', '5950', 9, 0.653), ('HCC84', '53081', 9, 0.326), ('HCC47', '5855', 9, 0.306), ('HCC2', 'V583', 9, 0.52), ('COPD_CARD_RESP_FAIL', '2989', 9, 0.42), ('HCC87', '78097', 9, 0.283), ('HCC106', '1101', 9, 1.313), ('HCC186', '7804', 9, 1.032), ('CANCER_IMMUNE', '8921', 9, 1.101), ('HCC75', '70705', 9, 0.281), ('HCC84', '27541', 9, 0.326), ('CANCER_IMMUNE', 'V4589', 9, 1.101)]


In [7]:
x = 0
y = 0
width = 40
height = 40
jane_coefs = indicator(jane,CC)  & CC.in_(cvars) & coefficient("CE_"+CC,Coef)
jane_coefs = sorted(jane_coefs,key= lambda val:  val[1], reverse=True)

normed = squarify.normalize_sizes([value for name, value in jane_coefs], width, height)
rects = squarify.squarify(normed, x, y, width, height)
shapes = []
annotations = []
counter = 0
for r in rects:
    name, value = jane_coefs[counter]
    shapes.append(
        dict(
            x0 = r['x'] + r['dx']/2 ,
            y0 = r['y'] + r['dy']/2 , 
            width = r['dx'],
            height = r['dy'],
            color=random.choice(Paired[12]),
            text=name,
            score=value,
            x1 = r['x'],
            y1 = r['y']
        ) 
    )
    counter = counter + 1

mapper = LinearColorMapper(palette=['#7fcdbb', '#41b6c4', '#1d91c0', '#225ea8', '#08589e'])

r_data = ColumnDataSource(ColumnDataSource.from_df(pd.DataFrame.from_dict(shapes)))
r_data.data['text'] = [rex.name_hcc(label) for label in r_data.data['text']]

q = figure(x_range=Range1d(x, width), y_range=Range1d(y, height), title='Condition Contribution to Jane Risk Score in Year 1')
q.rect( x='x0', y='y0', width='width', height='height', fill_color={'field':'score', 'transform':mapper},
           line_alpha=1, line_color='#FFFFFF', source=r_data)
q.text(x='x1', y='y1', text='text', source=r_data, x_offset=3, y_offset=1
       , text_font_size='8pt')
q.xaxis.visible = False
q.yaxis.visible = False





In [8]:

janealt_coefs = indicator(jane_alt,CC)  & CC.in_(cvars) & coefficient("CE_"+CC,Coef)
janealt_coefs = sorted(janealt_coefs,key= lambda val:  val[1], reverse=True)
normed_alt = squarify.normalize_sizes([value for name, value in janealt_coefs], width, height)
rects_alt = squarify.squarify(normed_alt, x, y, width, height)
shapes_alt = []

counter_alt = 0
for r in rects_alt:
    name, value = janealt_coefs[counter_alt]
    shapes_alt.append(
        dict(
            x0 = r['x'] + r['dx']/2 ,
            y0 = r['y'] + r['dy']/2 , 
            width = r['dx'],
            height = r['dy'],
            color=random.choice(Paired[12]),
            text=name,
            score=value,
            x1 = r['x'],
            y1 = r['y']
        ) 
    )
    counter_alt = counter_alt + 1

z_data = ColumnDataSource(ColumnDataSource.from_df(pd.DataFrame.from_dict(shapes_alt)))
z_data.data['text'] = [rex.name_hcc(label) for label in z_data.data['text']]

z = figure(x_range=Range1d(x, width), y_range=Range1d(y, height), title="Condition Contribution to Risk Score for Jane over multiple years")
z.rect( x='x0', y='y0', width='width', height='height', fill_color={'field':'score', 'transform':mapper},
           line_alpha=1, line_color='#FFFFFF', source=z_data)
z.text(x='x1', y='y1', text='text', source=z_data, x_offset=3, y_offset=1
       , text_font_size='8pt')
z.xaxis.visible = False
z.yaxis.visible = False






In [9]:
show(row(q,z))


## Scaling Up

So far we have been focused on doing this with only one patient but it is also possible for us to scale up the calculation (all the way to a full population). Here we will do 100 patients. 

In [10]:
cohort_study = %sql \
    SELECT d.PatientID, p.SEX, p.DOB \
        from diagnoses d join mem.patients p\
                on d.PatientID = p.PATIENT_ID\
        group by PatientID,p.SEX, p.DOB\
        having COUNT(*) > 20\
        order by Count(*) desc\
        limit 100;

Done.


In [11]:
def gender(l):
    if l == 'M':
        return 'male'
    else:
        return 'female'


cohort_patients = [Beneficiary(hicno=row[0], sex=gender(row[1]), dob=str(row[2])) for row in cohort_study[1:]]


In [12]:
hicno = [pat.hicno for pat in cohort_patients]
hicno = tuple(hicno)
diags = %sql SELECT DISTINCT PatientID, Diagnosis from diagnoses where PatientID in $hicno
diags = diags.DataFrame().set_index('PatientId')

for pat in cohort_patients:
    pat_diags = diags['Diagnosis'].loc[pat.hicno].tolist()
    for code in pat_diags:
        pat.add_diagnosis(Diagnosis(pat,code,ICDType.NINE))

Done.


Here is the average score for this cohort. 


In [13]:
cohort_score  = [score(pat,"community",Score)[0][0] for pat in cohort_patients[0:5]]
np.mean(cohort_score)

17.087599999999998

In [16]:
yt = []

for pat in cohort_patients:
    t = indicator(pat,CC)  & CC.in_(cvars) & coefficient("CE_"+CC,Coef)
    t = dict((x,y) for x, y in t)
    yt.append(t)
    


In [33]:
c = pd.DataFrame(yt)
c = c[[x for x in c.columns if x[0:2] == 'HC']].count()
c.name = 'PatCounts'
c.values

array([14,  4, 45, 18,  6, 32, 43, 24,  5,  2, 95,  1, 50, 22, 10,  3, 61,
       24,  6,  1,  3,  3,  6, 10,  9, 40, 12,  2,  6,  6, 16, 53, 46, 38,
       43, 40, 48, 58, 27,  6, 62, 36, 22, 64, 20, 23,  7, 53,  3, 18, 40,
       81, 81, 35, 12, 29, 56, 10, 12, 39, 47, 24,  7,  6, 18,  2,  4, 67,
        4, 15, 36, 54, 59, 12, 39,  4, 34, 99, 24, 49, 11, 31, 96, 26])

In [61]:
e = figure(x_range=list(c.index.values))
e_data = ColumnDataSource(dict(index=list(c.index.values), PatCounts=list(c.values)))
e.circle(source=e_data, x='index', y='PatCounts', size='PatCounts')
e.xaxis.axis_label_text_alpha = 0
e.xaxis.major_label_text_alpha = 0
e.grid.visible = False
show(e)